## 最適化ループ

このレッスンでは、*オプティマイザー*を使用して、ansatz のパラメータ化された量子状態を繰り返し探索する方法を学習します。

- 最適化ループのブートストラップ
- ローカルおよびグローバルオプティマイザーを使用する際のトレードオフの理解
- 「不毛な台地」の探索と回避の方法

大まかに言えば、オプティマイザーは探索空間を探索する中心です。オプティマイザーは、コスト関数の評価を使用して、変分ループで次のパラメーターセットを選択し、安定した状態に達するまで処理を繰り返します。この段階で、パラメーター値 $\vec\theta^*$ の最適なセットが返されます。

![最適化ワークフロー](images/optimization_workflow.png)

## ローカルおよびグローバルオプティマイザー

### ローカルオプティマイザー

ローカルオプティマイザーは、初期点 $C(\vec{\theta_0})$ から始まるコスト関数を最小化する点を探索し、連続する繰り返しにおいて現在評価している領域で観測したものに基づいて別の点に移動します。これらのアルゴリズムの収束は通常高速ですが、初期点に大きく依存する可能性があります。ローカルオプティマイザーは、現在評価している領域を超えた領域を見ることはできず、極小値に対してとりわけ脆い可能性があり、極小値を見つけると、より望ましい評価結果である他の状態を無視して収束したとみなしてしまいます。

### グローバルオプティマイザー

グローバルオプティマイザーは、ドメインのいくつかの領域 (つまりローカルではない) でコスト関数を最小化する点を探索し、オプティマイザーによって決定された $\Theta_i := \{{\vec\theta_{i,j} | j \in \mathcal{J}_\text{opt}^i} \}$ パラメータベクトルで反復的に (繰り返し $i$ ) 評価します。これにより極小値の影響を受けにくくなり、ある程度初期化に依存しなくなる一方で、提示される解への収束も大幅に遅くなります。

### ブートストラップ最適化

*ブートストラップ*、または事前の最適化に基づいてパラメーター $\vec\theta$  の初期値を設定すると、オプティマイザーがより速く解に収束するのに役立ちます。 これを初期点 $\vec\theta_0$ と呼び、 $|\psi(\vec\theta_0)\rangle = U_V(\vec\theta_0)|\rho\rangle$  を初期状態と呼びます。 この初期状態は参照状態 $|\rho\rangle$ とは異なります。前者は最適化ループ中に設定された初期パラメーターに焦点を当てているのに対し、後者は既知の「参照」解の使用に焦点を当てているためです。 $U_V(\vec\theta_0) \equiv I$ (つまり、恒等演算) の場合、これらは一致する可能性があります。

ローカルオプティマイザーが最適でない極小値に収束する場合、最適化をグローバルにブートストラップし、ローカルで収束を洗練することを試みることができます。これには2つの変分ワークロードを設定する必要がありますが、オプティマイザーがローカルオプティマイザーだけよりも最適な解を見つけることができるようになります。

## 勾配ありと勾配なしのオプティマイザー

### 勾配あり

コスト関数 $C(\vec\theta)$ において、初期点から関数 $\vec{\nabla} C(\vec\theta)$ の勾配にアクセスできる場合、関数を最小化する最も簡単な方法は、パラメータを関数の最急降下の方向に更新することです。 つまり、パラメータを $\vec\theta_{n+1} = \vec\theta_n - \eta \vec{\nabla} C(\vec\theta)$ として更新します。ここで、 $\eta$ は学習率 (更新のサイズを制御する小さな正の[ハイパーパラメータ](gloss:hyperparameter) ) です。 これをコスト関数の極小値 $C({\vec\theta^*})$ に収束するまで続けます。[`qiskit.algorithms`](https://qiskit.org/documentation/stubs/qiskit.algorithms.gradients.html) は、勾配を計算するためのいくつかの異なる方法を提供します。詳細については、[こちら](https://learn.qiskit.org/course/machine-learning/training-quantum-circuits#training-2-0)をご覧ください。

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

このコスト関数とオプティマイザーを使って、最適なパラメーターを算出することができます

In [ ]:
from qiskit.algorithms.optimizers import SPSA

initial_theta = np.ones(8)
optimizer = SPSA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

このタイプの最適化の主な欠点は、収束速度が非常に遅くなることと、最適解を達成する保証がないことです。

![theta に対する f(theta) のグラフ。複数の点は、曲線の最小値を見つける勾配降下法アルゴリズムのさまざまな状態を示します。](images/optimization_gradient_descent.png)

### 勾配なし

勾配を必要としない最適化アルゴリズムは、勾配情報を必要としないため、勾配の計算が困難であったり、コストがかかったり、ノイズが多すぎたりする場合に有効です。また、勾配を利用した手法が局所最適値に収束する傾向があるのに対して、大域最適値を見つける上でより頑健である傾向があります。ここでは、勾配なしオプティマイザーが不毛な台地を回避するのに役立ついくつかの例を紹介します。しかし、勾配なし手法は、特に高次元の探索空間を持つ問題に対してより高い計算リソースを必要とします。

ここでは、代わりに [`COBYLA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.COBYLA.html#qiskit.algorithms.optimizers.COBYLA) オプティマイザーを使用する例を紹介します：

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

## 不毛な台地

実際、コストの起伏は、下の例のように丘と谷のように非常に複雑になることがあります。最適化手法は、黒い点と線で示されるように、コストの起伏をナビゲートし、最小値を探します。3つの探索のうち2つは、グローバルな最小値ではなく、ローカルな最小値で終わっていることがわかります。

![コストの起伏](images/optimization_loss_landscape.png)

最適化手法の種類にかかわらず、コストの起伏が比較的平坦である場合、その手法が適切な探索方向を決定することは困難な場合があります。このようなシナリオは[不毛な台地](gloss:barren-plateaus)と呼ばれ、コストの起伏が徐々に平坦にります（したがって、最小値への方向性を決定するのがより困難になります）。パラメータ化された幅広い量子回路において、合理的な方向に沿った勾配が一定の精度でゼロでない確率は、量子ビットの数が増えるにつれて指数関数的に減少することが分かっています。

![Barren Plateaus](images/optimization_barren_plateaus.png)

この分野はまだ活発な研究が行われていますが、最適化性能を向上させるためのいくつかの推奨事項があります：

- **ブートストラップ** は、最適化ループが勾配が小さいパラメータ空間で立ち往生するのを避けるのに役立ちます。
- **ハードウェア効率の良いansatzの実験**: ブラックボックスオラクルとしてノイズの多い量子系を使用しているため、その評価の質はオプティマイザーの性能に影響を与えることがあります。[`EfficientSU2`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html) のようなハードウェア効率の良いansatzを使用することで、指数関数的に小さな勾配が発生することを回避できる可能性があります。
- **エラー抑制とエラー緩和の実験**: Qiskit Runtime Primitiveは、様々な `optimization_level`と `resilience_setting` をそれぞれ実験するためのシンプルなインターフェースを提供します。これにより、ノイズの影響を軽減し、最適化プロセスをより効率的に行うことができます。
- **勾配なしオプティマイザーを使った実験**: `COBYLA`のようなオプティマイザーは、勾配ベースの最適化アルゴリズムとは異なり、勾配情報に頼らずにパラメータを最適化するため、不毛な台地の影響を受けにくいです。

このレッスンで、あなたは最適化ループを定義する方法を学びました：

- 最適化ループをブートストラップする。
- ローカルオプティマイザーとグローバルオプティマイザーを使い分けながら、トレードオフを理解する。
- 不毛の台地とそれを避ける方法を探る

ハイレベルの変分ワークロードは完了です：

![最適化回路](images/optimization_circuit.png)

次に、このフレームワークを意識して、具体的な変分アルゴリズムを探っていきます。